In [1]:
import os
import pickle
import time
import random

import torch

import numpy as np

from konlpy.tag import Twitter

from tqdm import tqdm

from seq2seq.dataset import Dataset
from seq2seq.model import Seq2seqModel
from seq2seq.trainer import Trainer

In [2]:
USE_CUDA = torch.cuda.is_available()
GPU_ID = 1

USE_CUDA

True

# 1. Preprocess Dataset

In [3]:
SRC_FILE_PATH = 'dataset/korean_dialog/hangul_src.txt'
TGT_FILE_PATH = 'dataset/korean_dialog/hangul_tgt.txt'

EXPR_PATH = 'experiment/'

DATASET_PKL = EXPR_PATH + 'dataset.pkl'
MODEL_PKL = EXPR_PATH + 'test_model10.model'

BATCH_SIZE = 512
MAX_LENGTH = 30

In [4]:
tagger = Twitter()

In [5]:
if os.path.isfile(DATASET_PKL):
    with open(DATASET_PKL, 'rb') as fp:
        dataset = pickle.load(fp)
else:
#    start_time = time.time()
    dataset = Dataset(
        src_file_path=SRC_FILE_PATH,
        tgt_file_path=TGT_FILE_PATH,
        max_length=MAX_LENGTH
    )
    dataset.prepareData(tagger=tagger)
#    print("--- %s seconds ---" % (time.time() - start_time))
    with open(DATASET_PKL, 'wb') as fp:
        pickle.dump(dataset, fp)

# 2. Define Model

In [6]:
INPUT_SIZE = dataset.src_vocab_size
HIDDEN_SIZE = 1024
OUTPUT_SIZE = dataset.tgt_vocab_size

In [7]:
model = Seq2seqModel(
    name='test_model',
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,
    output_size=OUTPUT_SIZE,
    max_length=MAX_LENGTH,
    gpu_id=GPU_ID
)

# 3. Train Model

In [8]:
trainer = Trainer(
    model=model,
    dataset=dataset,
    gpu_id=GPU_ID,
    print_interval=10,
    plot_interval=10,
    checkpoint_interval=10,
    expr_path=EXPR_PATH
)

In [9]:
#trainer.train(num_epoch=200, batch_size=512)

# 4. Test Model

In [10]:
def modelTest(num, model_epoch, beam_size):
    model.load_state_dict(torch.load(EXPR_PATH+'test_model'+str(model_epoch)+'.model'))
    
    # sample 테스트
    for i in range(num):
        test_pair = random.choice(dataset.test_pairs)
        print('> ', ' '.join(test_pair[0]))
        print('< ', ' '.join(model.sampleResponce(test_pair[0], tagger, dataset.src_vocab, dataset.tgt_vocab, beam_size=beam_size)[1:-1]))
        print()

In [11]:
modelTest(num=20, model_epoch=150, beam_size=1)

>  어떻 게 생긴 물건 이 죠
<  여기 까지 오신 다고 합 니까

>  예약한 좌석 을 확인 부탁드립니 다
<  네 무슨 일 이 시 죠

>  실례합 니다 물 한 잔 더 주실 수 있 을까요
<  이 곳 에서 맥주 를 마실 수 있 나요

>  제 가 주문한 음식 이 이 음식 이 맞 습니까
<  무엇 을 도 와 드릴 까 요

>  그 방 혹시 청소 가 안 되어 있 나요
<  한 장 얻 을 수 있 을 거 예요

>  택시 비는 얼마나 할 까 요
<  찾아오는 데 어렵 지는 않으셨나 요

>  음 라오스 를 가고 싶네 요 어떻 게 가야 하나요
<  배 안 에서 영어 로 안내 방송 이 나옵 니까

>  버스정류장 은 걸어 서 10 분 정도 거리 에 있 습니다
<  버스정류장 은 걸어 서 10 분 정도 거리 에 있 습니다

>  그렇 군요 그럼 그 걸 로 하겠 습니다
<  그런 가요 그냥 이 걸 로 주세 요

>  물론 이에요
<  물론 이에요

>  성인 2 장 이 요
<  네 즐기는 편입 니 다

>  네 맞아 요 카메라 로 보니 새롭 네요
<  네 여기 가 디자인 이 아주 예쁘 네요

>  그렇 겠 군요 알 겠 습니다 지불하겠 습니다
<  20 분 정도 기다리 셔 야 합 니다

>  시간 은 정확히 얼마나 걸리는 지는 모르 겠 네요
<  주의하겠 습니다 여기 5 위안 더 드리 겠 습니다

>  아니 요 고구마 그라탕 과 피자 만 포장 해 주세 요
<  이 근처 에 왜 그러 신지 요 지금 같이 가 주 세요

>  준비해야 할 것 은 있 나요
<  아 무엇 이 필요하신 가요

>  내일 오후 에 이용하고 싶 습니다
<  이 음료 는 무엇 인가요

>  속도 를 위반하셨 습니다 티켓 을 드려 야 겠 습니다
<  제 가 확인해 봤 을 때 는 없었 는데 이상하네 요

>  실내 에는 처음 들어오 자마자 보이는 화장실 뿐 이에요
<  네 오늘 밤 예약 을 하고 싶 습니다

>  당연하죠
<  네

